# The Ratcliffe-Tschantz $5$-manifold

> **_Note:_** This notebook attempts to compute the equivariant chain complex of the Ratcliffe-Tschantz manifold,\
but the computation takes an extremely long time (at least 10 hours) and has never been completed. 

In [1]:
import twisted_l2
import regina
from twisted_l2.regina_bary_sub import bary_sub # barycentric subdivision for 5-manifolds

We construct a triangulation from a Regina _isomorphism signature_:

In [2]:
twisted_l2.isosig_RT5

'-cadvvAMzzLzvAvvLvvvzvzLPQvzLvzMLvQvALvLQQMQAwPwwLAQwMQQwMPQAQMwQPwQMAQPMAQQAMQvwPMzMvQPMPvPQPvvQPQAzvAMwQQvQQMQAQQQLwQQvzQwwLQwQQQLQzQQQAzQQAQAQQQQQQPQLMQQQQQPQQQPQQQQQQLQQQQQMQQPQQQQQAQQPQPQQQQQcadafadafafakajaoajazaIaOavaFauasataGaEaZa3atauauaTaUaVaEa9aLa0a5acbfbjb5aBaCaDadbCaOaDaLaDadbeb9aMa1aIaJaKahbJaKaKa8albhbgbibsbtbubBb2aCbzbFbQaRaqbSaRa3aSa2aSawbabUaVaAbVaxbXaYaZaLbYaZa+aZaybsblbmbkbvbxbwbDbBbEbQbRbPb5a6a7a6a7aqb7atbdbebZbwbHbzbHbdbnbObMbnbWbJbUbebCbRbhbAbib+biblbmbLbmbZbdcgc9b3bKbhcGbncWbVbPbVbTbtb+bububwbxbZbYbxbvcEb-b+b2b3b4bwcvcEbvcDbacRbbcEbdclcecpcocVbqcObtcCc9b8bHcUbkcNc7bWbObDcGcNc7bWbscTbtczcFcbcacQcicocsctcTc9b8bVb7b7bcc-b0bAcyc5bucVccc-bUcDcucLc6btc3b4bUc4b5bTcScWcycAcXcLcuc9brcTc9bQcAcccUcTcbcocUcvc1c0cBc0cCclcgcHchcicGc0chcicicscrcScmcEcNcJcKcIcRcBcCcEcScrcOcRc4cscOcpcqcHcqcCcNcLcEcSctcLcXc2cxcMcMc2c6cZcWcAc5cFcRcQc0cWcCcYcOcPcGcScKcQcRcOcPcKc8cNcJcKcMcYcKcWcXc6c2cPcYc6cRc3c7c1cZc5c9c9c+c9c7c1c5c7c5c6c-c-c8c+c4c-c8c+c7c9c9c+c-c-c4bieaiyaqboagfgfgfgfaagfaaaagfIe0

It can be obtained by running the [script](https://github.com/topologia-pisa/tessellated-manifold-triangulation)
provided with the Italiano-Martelli-Migliorini paper.

In [3]:
tri = regina.Triangulation5.fromIsoSig(twisted_l2.isosig_RT5)

In [4]:
tri.countVertices()

5

In [5]:
%time tri_sd = bary_sub(tri)

CPU times: user 1.33 s, sys: 33.1 ms, total: 1.37 s
Wall time: 1.37 s


In [6]:
tri_sd.fVector()

[1800, 40198, 215040, 453120, 414720, 138240]

### Determining which vertices are ideal

If you have the file `tessellation.py` from the same [repository](https://github.com/topologia-pisa/tessellated-manifold-triangulation),
you can place it in the same directory as this notebook\
and use it to find the link of every vertex in `tri`:

In [7]:
%%time
from tessellation import get_link
links = []
for i in range(tri.countVertices()):
    links.append(get_link(tri.vertex(i)))

CPU times: user 68 ms, sys: 3.43 ms, total: 71.5 ms
Wall time: 71.7 ms


In [8]:
%%time
for j, lk in enumerate(links):
    print(f"{j}: {lk.fundamentalGroup().abelianisation().detail().strip()}")

0: Z + Z_4
1: 0
2: 0
3: Z + 2 Z_4
4: 0
CPU times: user 120 ms, sys: 8 µs, total: 120 ms
Wall time: 120 ms


The only nontrivial links correspond to ideal vertices (0 and 3), whose cusps have homology groups
$\mathbb Z \oplus \mathbb Z_4$ and $\mathbb Z \oplus \mathbb Z_4^2$, respectively.

## Making the CW complex

From the documentation of Regina's barycentric subdivision function (which is extended by `bary_sub`):

    Each top-dimensional simplex s is divided into (dim + 1) factorial sub-simplices by placing
    an extra vertex at the centroid of every face of every dimension.
    Each of these sub-simplices t is described by a permutation p of (0, ..., dim).
    The vertices of such a sub-simplex t are:

        vertex p[0] of s;
        the centre of edge (p[0], p[1]) of s;
        the centroid of triangle (p[0], p[1], p[2]) of s;
        ...
        the centroid of face (p[0], p[1], p[2], p[dim]) of s, which is the entire simplex s itself.

    The sub-simplices have their vertices numbered in a way that mirrors the original simplex s:

        vertex p[0] of s will be labelled p[0] in t;
        the centre of edge (p[0], p[1]) of s will be labelled p[1] in t;
        the centroid of triangle (p[0], p[1], p[2]) of s will be labelled p[2] in t;
        ...
        the centroid of s itself will be labelled p[dim] in t.

In [9]:
def get_subdiv_vertex(tri_sd, f, i):
    """Returns the image of vertex i of simplex f in the barycentric subdivision tri_sd"""
    vert, = {tri_sd.simplex(j).vertex(i).index() for j in range(720*f + 120*i, 720*f + 120*(i+1))}
    return vert

In [10]:
for spx in range(10):
    # the vertices of a simplex
    vertT = [tri.simplex(spx).vertex(i).index() for i in range(6)]
    # are mapped to these vertices in the barycentric subdivision
    vertS = [get_subdiv_vertex(tri_sd, spx, i) for i in range(6)]
    print(vertT, vertS)

[0, 0, 0, 1, 0, 2] [0, 0, 0, 49, 0, 54]
[3, 0, 0, 1, 0, 2] [55, 0, 0, 49, 0, 54]
[0, 3, 0, 1, 0, 2] [0, 55, 0, 49, 0, 54]
[0, 0, 3, 1, 0, 2] [0, 0, 55, 49, 0, 54]
[0, 0, 0, 3, 0, 2] [0, 0, 0, 55, 0, 54]
[0, 0, 0, 1, 3, 2] [0, 0, 0, 49, 55, 54]
[0, 0, 0, 1, 0, 2] [0, 0, 0, 49, 0, 54]
[3, 0, 0, 0, 0, 2] [55, 0, 0, 0, 0, 54]
[3, 0, 0, 1, 0, 4] [55, 0, 0, 49, 0, 218]
[0, 3, 0, 0, 0, 2] [0, 55, 0, 0, 0, 54]


Vertices 0, 1, 2, 3, 4 correspond to 0, 49, 54, 55, 218 in `tri_sd`.

Hence, we construct a regular CW complex by taking all the cells of `tri_sd` not containing the vertices 0 or 55.

In [11]:
%%time

def is_good_face(f):
    return all(f.vertex(j).index() not in (0,55) for j in range(f.subdimension + 1))

def all_verts(f):
    return [f.vertex(j).index() for j in range(f.subdimension + 1)]

index_to_id = [{} for i in range(6)]
fv = tri_sd.fVector()
fl = [[[1,0] for i in range(fv[0] - 2)]] # remove the two ideal vertices

for i in range(fv[0]):
    if i not in (0,55):
        index_to_id[0][i] = len(index_to_id[0])

for d in range(1,5):
    fd = []
    for spx in tri_sd.faces(d):
        if is_good_face(spx):
            fd.append([d+1] + [index_to_id[d-1][spx.face(d-1, i).index()]+1 for i in range(d+1)])
            index_to_id[d][spx.index()] = len(index_to_id[d])
    fl.append(fd)

fl.append([])
for k in range(fv[5]):
    if is_good_face(tri_sd.simplex(k)):
        fl[5].append([5+1] + [index_to_id[5-1][tri_sd.simplex(k).face(5-1, i).index()]+1 for i in range(5+1)])
        index_to_id[5][k] = len(index_to_id[5])

fl.append([])

CPU times: user 9.33 s, sys: 106 ms, total: 9.43 s
Wall time: 9.45 s


In [12]:
twisted_l2.load_hap()

true

In [13]:
%time cw = twisted_l2.cw_complex(fl)

CPU times: user 12.3 s, sys: 106 ms, total: 12.4 s
Wall time: 12.3 s


In [14]:
[twisted_l2.gap_member(cw, "nrCells")(j) for j in range(6)]

[1798, 35158, 151920, 251040, 174720, 42240]

In [15]:
%time scw = twisted_l2.simplify_cw(cw)

CPU times: user 8min 51s, sys: 3.24 s, total: 8min 54s
Wall time: 8min 51s


In [16]:
[twisted_l2.gap_member(scw, "nrCells")(j) for j in range(6)]

[1798, 29580, 86729, 88228, 31296, 2015]

In [17]:
%time chain_complex = twisted_l2.equivariant_cc(scw)

CPU times: user 6min 50s, sys: 2.34 s, total: 6min 52s
Wall time: 6min 50s


In [18]:
chain_complex.group

<fp group on the generators [ f1, f2 ]>

In [19]:
[chain_complex.dimension(i) for i in range(6)]

[1, 2, 104, 427, 324, 0]

In [20]:
len(chain_complex.elts)

4520

This is the number of group elements appearing in the matrices of the chain complex.

In [21]:
G = twisted_l2.get_fundamental_group(chain_complex)

In [22]:
%time diff1 = twisted_l2.boundary_operator(chain_complex, 1, G); diff1

CPU times: user 164 ms, sys: 12 µs, total: 164 ms
Wall time: 164 ms


[1 - F[0]]
[1 - F[1]]

In [23]:
%time diff2 = twisted_l2.boundary_operator(chain_complex, 2, G); diff2

CPU times: user 787 ms, sys: 5.16 ms, total: 792 ms
Wall time: 796 ms


104 x 2 dense matrix over Algebra of Free group indexed by {0, 1} over Rational Field (use the '.str()' method to see the entries)

In [24]:
twisted_l2.configs.LogOptions.LEVEL = twisted_l2.INFO

The `ask` parameter makes the function interactive, so we can observe the exact row at which it freezes.

You can press Enter to proceed to the next row, or type "n" and press Enter to stop.

In [25]:
diff3 = twisted_l2.boundary_operator(chain_complex, 3, G, ask = True)

Next row? (Y/n)  


Extracted boundary row #0 of length 0


Next row? (Y/n)  


Extracted boundary row #1 of length 0


Next row? (Y/n)  


...

Next row? (Y/n)  


Extracted boundary row #8 of length 765


Next row? (Y/n)  


Extracted boundary row #9 of length 0


KeyboardInterrupt: Interrupted by user

This is the exact computation on which it freezes:

In [26]:
chain_complex.boundary(3, 11)

KeyboardInterrupt: 